In [69]:
import asyncio
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
nest_asyncio.apply()
import os
from anthropic import AsyncAnthropic
from xml.sax.saxutils import escape
import re
import json

In [70]:
prompt = """\n\n
The above is a transcript of a lecture video with timestamps for each sentence in seconds.

Your task is to create a 30 second engaging and educational tiktok script for one topic in the video. 
Choose one of the more obscure and interesting topics from the transcript that most people dont know about.
The tiktok should incorporate an engaging story or example.
Do not have any emojis or hashtags in the script.
The script should be in ssml format. But do not change voices, only add pauses and emphasis.
"""

music_prompt = """
The above is a script for a tiktok video.
Please generate a short one sentence description of the music that should be playing in the background of the video.
Include genre and mood
Example:
"A short upbeat EDM tune with a catchy melody"
"""

In [71]:
# Load transcript from data/transcript.txt
with open("data/transcript.txt", "r") as f:
    transcript = f.read()

In [72]:
async def generate_text(text, script):
    client = AsyncAnthropic(
        # This is the default and can be omitted
        api_key=os.environ.get("ANTHROPIC_API_KEY"),
    )

    message = await client.messages.create(
        temperature=0,
        max_tokens=512,
        messages=[
            {
                "role": "user",
                "content": text + script,
            },
            {"role": "assistant", "content": "<"},
        ],
        model="claude-3-opus-20240229",
    )

    return message.content

response = asyncio.run(generate_text(transcript, prompt))

In [73]:
script = f"<{response[0].text}"

In [74]:
print(script)

<speak>
Have you ever heard of a <emphasis>trie</emphasis>? No, not the verb "try", I'm talking about the <emphasis>data structure</emphasis> called a trie, spelled T-R-I-E. <break time="1s"/>

Imagine you're organizing a massive library, but instead of books, you're dealing with <emphasis>words</emphasis>. A trie is like a tree of arrays that allows you to store and retrieve these words at lightning-fast speeds. <break time="2s"/>

Picture each letter of the alphabet having its own shelf. To store a word, you simply place it on the shelf corresponding to its first letter. Then, <emphasis>each subsequent letter</emphasis> gets its own sub-shelf branching off from the previous one. <break time="2s"/>

The magic happens when you're looking for a specific word. <emphasis>No matter how many words are in the library, you can find any word in the same amount of time</emphasis> - just by following the path created by its letters! <break time="2s"/>

So the next time you're typing on your phon

In [75]:
async def generate_text(text, script):
    client = AsyncAnthropic(
        # This is the default and can be omitted
        api_key=os.environ.get("ANTHROPIC_API_KEY"),
    )

    message = await client.messages.create(
        temperature=0,
        max_tokens=64,
        messages=[
            {
                "role": "user",
                "content": text + script,
            },
        ],
        model="claude-3-opus-20240229",
    )

    return message.content


response = asyncio.run(generate_text(script, music_prompt))
music = response[0].text

In [76]:
music

'Playful and quirky instrumental background music with a light, whimsical tone.'

In [77]:
with open("data/data.json", "w") as f:
    json.dump({"script": script, "music_description": music}, f, indent=4)